This part mainly contains the basic attention mechanism and a simple transformer-based model, the gpt2 model.

> some codes referenced from <a href='https://scholar.harvard.edu/binxuw/classes/machine-learning-scratch/materials/transformers'>MLFS</a>

# Self-Attention: Single Head

In [1]:
!pip install einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.9 MB/s eta 0:00:00


In [10]:
import torch
from torch import nn
import torch.nn.functional as F
import numpy as np
import math
from einops import rearrange

In [11]:
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

In [57]:
class SingleHeadAttention:
    def __init__(self, emb_dim, head_dim, head=1):
        self.to_q = # your code here
        self.to_k = # your code here
        self.to_v = # your code here

    def get_qkv(self, tokens):
        # your code here
        return q, k, v

    def __call__(self, tokens):
        # your code here
        return val

In [58]:
# TEST implementation
emb_dim = 128
head_dim = 64
tokens = torch.rand(32, 12, emb_dim)

m = SingleHeadAttention(emb_dim, head_dim)
q, k, v = m.get_qkv(tokens)
assert torch.allclose(m(tokens), F.scaled_dot_product_attention(q, k, v))

# Self-Attention: Multi Head

In [103]:
class MultiHeadAttention:
    def __init__(self, emb_dim, head_dim, head):
        assert emb_dim == head_dim * head, "required! emb_dim == head_dim * head"

        inner_dim = head_dim * head
        self.head_dim = head_dim
        self.head = head
        self.to_qs = # your code here
        self.to_ks = # your code here
        self.to_vs = # your code here

    def get_qkv(self, tokens):
        bs, t, _ = tokens.shape
        # your code here
        # ....
        
        assert q.shape == (bs, self.head, t, self.head_dim)
        assert k.shape == (bs, self.head, t, self.head_dim)
        assert v.shape == (bs, self.head, t, self.head_dim)
        return q, k, v

    def get_att_score(self, tokens):
        # your code here

    def __call__(self, tokens):
        # your code here

In [104]:
# TEST implementation
emb_dim = 768
head_dim = 64
head = 12

tokens = torch.rand(32, 12, emb_dim)

m = MultiHeadAttention(emb_dim, head_dim, head)
mha = nn.MultiheadAttention(emb_dim, head, batch_first=True,)

# load state dict
tmp_dict = {
    'in_proj_weight': torch.cat([m.to_qs.weight.data.T, m.to_ks.weight.data.T, m.to_vs.weight.data.T], dim=1).T
}
for k, v in mha.state_dict().items():
    if k in tmp_dict:
        continue
    tmp_dict[k] = v
mha.load_state_dict(tmp_dict)

mha_att = mha(tokens, tokens, tokens, average_attn_weights=False)[1]

assert torch.allclose(m.get_att_score(tokens), mha_att, atol=1e-6, rtol=1e-6)

# Multi Head Attention with mask

In [132]:
class MultiHeadAttentionMask:
    def __init__(self, emb_dim, head_dim, head, token_num):
        assert emb_dim == head * head_dim
        inner_dim = head * head_dim

        self.head = head
        self.head_dim = head_dim

        self.to_qs = # your code here
        self.to_ks = # your code here
        self.to_vs = # your code here

        self.mask = # your code here

    def get_qkv(self, tokens):
        bs, t, _ = tokens.shape
        # your code here

        assert q.shape == (bs, self.head, t, self.head_dim)
        assert k.shape == (bs, self.head, t, self.head_dim)
        assert v.shape == (bs, self.head, t, self.head_dim)
        return q, k, v

    def get_att(self, tokens):
        # your code here

    def __call__(self, tokens):
        # your code here

In [133]:
# TEST implementation
emb_dim = 768
head_dim = 64
head = emb_dim // head_dim
token_num = 12

tokens = torch.rand(32, token_num, emb_dim)

m = MultiHeadAttentionMask(emb_dim, head_dim, head, token_num)
mha = nn.MultiheadAttention(emb_dim, head, batch_first=True)

# load state dict
tmp_dict = {
    'in_proj_weight': torch.cat([m.to_qs.weight.data.T, m.to_ks.weight.data.T, m.to_vs.weight.data.T], dim=1).T
}
for k, v in mha.state_dict().items():
    if k in tmp_dict:
        continue
    tmp_dict[k] = v
mha.load_state_dict(tmp_dict)

mha_att = mha(tokens, tokens, tokens, average_attn_weights=False, attn_mask=m.mask)[1]
m_att = m.get_att(tokens)

assert torch.allclose(m_att, mha_att, atol=1e-6, rtol=1e-6)

# Transformer Model

Key Part in Transformer Model

- Res connection
- LayerNorm
- Multi-Head Attention
- FFN

Using gpt2 as an exmaple to impelment a simple Transformer model. Arch:
1. word token embedding (50257, 768)
2. word position embedding (1024, 768)
3. 12 * gpt2_transformer_block
4. layer norm

Arch of gpt2 transformer block
```
     x-----------
layer_norm      |
mul_head_att    |
  res_add ------|
     x-----------
layer_norm      |
   ffn          |
 res_add -------|
```

In [ ]:
!pip install transformers[torch]

In [22]:
class GPT2Block:
    def __init__(self, emb_dim, head, drop_out=0.0):
        self.ln1 = # your code here
        self.attn = # your code here
        self.ln2 = # your code here
        self.ffn = # your code here

    def get_causal_mask(self, token_num):
        # your code here
        return mask

    def __call__(self, tokens, is_causal=True):
        # your code here

**TEST implementation**

In [9]:
# @title test utils

from transformers.activations import NewGELUActivation

def GPT2block_to_TransformerBlock_simple(tfmblock, gpt2block, ):
    """copy the weights from a GPT2 block to a TransformerBlock_simple"""
    tfmblock.ln1.weight.data = gpt2block.ln_1.weight
    tfmblock.ln1.bias.data = gpt2block.ln_1.bias
    tfmblock.ln2.weight.data = gpt2block.ln_2.weight
    tfmblock.ln2.bias.data = gpt2block.ln_2.bias
    tfmblock.attn.in_proj_weight.data = gpt2block.attn.c_attn.weight.T
    tfmblock.attn.in_proj_bias.data = gpt2block.attn.c_attn.bias
    tfmblock.attn.out_proj.weight.data = gpt2block.attn.c_proj.weight.T
    tfmblock.attn.out_proj.bias.data = gpt2block.attn.c_proj.bias
    tfmblock.ffn[0].weight.data = gpt2block.mlp.c_fc.weight.T
    tfmblock.ffn[0].bias.data = gpt2block.mlp.c_fc.bias
    tfmblock.ffn[1] = NewGELUActivation()
    # mlp in GPT2 and BERT used a new GELU activation, using nn.GeLU() will cause a small error around 1E-3
    tfmblock.ffn[2].weight.data = gpt2block.mlp.c_proj.weight.T
    tfmblock.ffn[2].bias.data = gpt2block.mlp.c_proj.bias
    return tfmblock

def GPT2Model_to_GPT2Model_simple(gpt2modelsimple, gpt2model, ):
    """copy the weights from a GPT2 model to a GPT2Model_simple"""
    gpt2modelsimple.wte.weight.data = gpt2model.wte.weight
    gpt2modelsimple.wpe.weight.data = gpt2model.wpe.weight
    gpt2modelsimple.ln_f.weight.data = gpt2model.ln_f.weight
    gpt2modelsimple.ln_f.bias.data = gpt2model.ln_f.bias
    for i in range(12):
        GPT2block_to_TransformerBlock_simple(gpt2modelsimple.blocks[i], gpt2model.h[i])
    return gpt2modelsimple

In [5]:
from transformers import GPT2Tokenizer, GPT2Model

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2Model.from_pretrained("gpt2")
model.eval()

GPT2Model(
  (wte): Embedding(50257, 768)
  (wpe): Embedding(1024, 768)
  (drop): Dropout(p=0.1, inplace=False)
  (h): ModuleList(
    (0-11): 12 x GPT2Block(
      (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): GPT2Attention(
        (c_attn): Conv1D()
        (c_proj): Conv1D()
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): GPT2MLP(
        (c_fc): Conv1D()
        (c_proj): Conv1D()
        (act): NewGELUActivation()
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
)

In [23]:
embdim = 768
headcnt = 12
tfmblock = GPT2Block(embdim, headcnt)
GPT2block_to_TransformerBlock_simple(tfmblock, model.h[0])

tokens_embs = torch.randn(2, 5, 768)
tfmblock_out = tfmblock(tokens_embs, is_causal=True)
modelblock_out, = model.h[0](tokens_embs)
assert torch.allclose(tfmblock_out, modelblock_out, atol=1e-5, rtol=1e-5)

**Simple GPT2 Model**

In [32]:
class GPT2:
    def __init__(self, emb_dim, head, drop_out=0.0):
        self.wte = # your code here
        self.wpe = # your code here
        self.blocks = # your code here
        self.ln_f = # your code here

    def __call__(self, input_ids, is_causal=True):
        # your code here

In [33]:
embdim = 768
headcnt = 12

model_ours = GPT2(embdim, headcnt)
GPT2Model_to_GPT2Model_simple(model_ours, model)

inputs = tokenizer("I have a cat, her name is", return_tensors="pt")
outputs = model(**inputs, )

hidden_last_ours = model_ours(inputs['input_ids'])

assert torch.allclose(outputs.last_hidden_state, hidden_last_ours, atol=1e-5, rtol=1e-5)